# Satellite Location Demo

Timeliness is an important aspect for countless uses of Earth Observation (EO) data. Humanitarians, for example, benefit from imagery immediately before or after an emergency.

Knowing when an image was captured of an area of interested, or when it will be captured next, can help humanitarians better understand where aid is most needed.

This notebook demonstrates how the location of EO satellites can be computed, both in the past and in the future, in an area of interest. This can be used as a proxy of where images were and will be captured. 

These techniques could be combined with those in other notebooks in this repository, such as loading STAC items and calculating how a disaster impacts people, to create automated analysis for past and future disasters.

## Load Libraries and Data

We'll use the [sgp4](https://pypi.org/project/sgp4/) and [skyfield](https://rhodesmill.org/skyfield/) libraries, which help us calculate the position of satellites using complex orbital physics.

In [49]:
from skyfield.api import load, wgs84, Timescale

from datetime import datetime, timedelta, timezone

from lonboard import viz, Map, ScatterplotLayer, basemap, PathLayer
from lonboard.colormap import apply_continuous_cmap
from palettable.colorbrewer.sequential import Purples_9
from palettable.colorbrewer.diverging import BrBG_9

import geopandas as gpd
import pandas as pd
from shapely import Point, LineString

We'll load information about specific satellites from [Celestrak](https://celestrak.org). Due to the complexities of orbital physics, data about satellites needs to be updated frequently is only accurate for about 1 week before and after "epoch".

For this example, we'll calculate the position of Sentinel 2b.

In [2]:
sentinel_2b_url = "https://celestrak.org/NORAD/elements/gp.php?CATNR=42063"

sentinel_2b = load.tle_file(sentinel_2b_url, reload=True, filename="sentinel_2b")

[#################################] 100% sentinel_2b


In [3]:
satellite = sentinel_2b[0]
satellite

<EarthSatellite SENTINEL-2B catalog #42063 epoch 2024-07-04 04:48:48 UTC>

---

## Calculating the position of a satellite

For sudden-onset emergencies, the time immediately before an after an event are the most critical. We can set a time range of +- 1 day from a certain point in time. For the purposes of this notebook, we'll use `now` (the time when the cell is run).

In [4]:
# Calculate the current UTC time (without microseconds), then creating a time range +/- 24h

# load timescale
ts = load.timescale()

# load "ephemeris" (https://rhodesmill.org/skyfield/api.html#planetary-ephemerides)
eph = load('de421.bsp')

now = datetime.now(timezone.utc).replace(microsecond=0)
t1 = now
t2 = now + timedelta(days=2)

To calculate the position of the satellite throughout the time range, we can initiate a timer and then calculate the position at each time step (defined below).

In [5]:
# initiate

timer = t1

df = pd.DataFrame(columns=['satellite', 'timestamp', 'coordinates', 'lng', 'lat', 'daytime'])

rows = []

The `location_iteration` function lodes the geocentric location, calculates the latitude and longitude, converts them to decimal degrees, and saves them as a point coordinate. It also checks if the satellite is sunlit at the time of calculation, which can be used as an approximation of if it is daytime below.

In [6]:
def location_iteration(timer, sat):
    geocentric = sat.at(Timescale.from_datetime(ts, timer))
    lat, lon = wgs84.latlon_of(geocentric)
    longitude = lon.degrees
    latitude = lat.degrees
    daytime = geocentric.is_sunlit(eph)
    coords = Point(longitude, latitude)

    return timer, longitude, latitude, coords, daytime

Then, we can iterate over the time frame. The more frequent the measurements, the slower the calculation takes.

In [7]:
while timer <= t2:
    timer, longitude, latitude, coords, daytime = location_iteration(timer, sentinel_2b[0])

    row = pd.DataFrame({'satellite': satellite.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
    rows.append(row)    

    timer += timedelta(seconds=15) # ~13 seconds

df = pd.concat(rows, ignore_index=True)
df["time_string"] = df["timestamp"].dt.strftime('%Y-%m-%d %X')

We can then save this as a geodataframe.

In [8]:
sentinel_2b_gdf = gpd.GeoDataFrame(df, geometry="coordinates")
sentinel_2b_gdf

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
0,SENTINEL-2B,2024-07-04 14:47:19+00:00,POINT (118.85968 -20.19238),118.859682,-20.192376,False,2024-07-04 14:47:19
1,SENTINEL-2B,2024-07-04 14:47:34+00:00,POINT (118.64705 -19.30638),118.647049,-19.306379,False,2024-07-04 14:47:34
2,SENTINEL-2B,2024-07-04 14:47:49+00:00,POINT (118.43601 -18.42013),118.436012,-18.420125,False,2024-07-04 14:47:49
3,SENTINEL-2B,2024-07-04 14:48:04+00:00,POINT (118.22648 -17.53363),118.226476,-17.533625,False,2024-07-04 14:48:04
4,SENTINEL-2B,2024-07-04 14:48:19+00:00,POINT (118.01835 -16.64689),118.018350,-16.646890,False,2024-07-04 14:48:19
...,...,...,...,...,...,...,...
11516,SENTINEL-2B,2024-07-06 14:46:19+00:00,POINT (-65.92535 -12.31391),-65.925352,-12.313913,True,2024-07-06 14:46:19
11517,SENTINEL-2B,2024-07-06 14:46:34+00:00,POINT (-66.12781 -13.20165),-66.127811,-13.201648,True,2024-07-06 14:46:34
11518,SENTINEL-2B,2024-07-06 14:46:49+00:00,POINT (-66.33127 -14.08920),-66.331275,-14.089199,True,2024-07-06 14:46:49
11519,SENTINEL-2B,2024-07-06 14:47:04+00:00,POINT (-66.53582 -14.97656),-66.535820,-14.976557,True,2024-07-06 14:47:04


Let's refine our area of interest to see when/where satellites pass overhead. We can use a large bounding box of South Asia for demonstration purposes.

In [9]:
# Florida

min_lon = -89
max_lon = -74
min_lat = 22
max_lat = 32

bbox=[min_lon, max_lon, min_lat, max_lat]
bbox

[-89, -74, 22, 32]

In [10]:
mask_lon = (sentinel_2b_gdf.lng >= min_lon) & (sentinel_2b_gdf.lng <= max_lon)
mask_lat = (sentinel_2b_gdf.lat >= min_lat) & (sentinel_2b_gdf.lat <= max_lat)

aoi = sentinel_2b_gdf.where(mask_lon & mask_lat).dropna()
aoi

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
6230,SENTINEL-2B,2024-07-05 16:44:49+00:00,POINT (-88.35006 31.76702),-88.350062,31.767024,True,2024-07-05 16:44:49
6231,SENTINEL-2B,2024-07-05 16:45:04+00:00,POINT (-88.59483 30.88337),-88.594829,30.883369,True,2024-07-05 16:45:04
6232,SENTINEL-2B,2024-07-05 16:45:19+00:00,POINT (-88.83629 29.99934),-88.836289,29.999342,True,2024-07-05 16:45:19
8910,SENTINEL-2B,2024-07-06 03:54:49+00:00,POINT (-84.79976 22.86861),-84.799764,22.868613,False,2024-07-06 03:54:49
8911,SENTINEL-2B,2024-07-06 03:55:04+00:00,POINT (-85.02014 23.75511),-85.020137,23.755105,False,2024-07-06 03:55:04
8912,SENTINEL-2B,2024-07-06 03:55:19+00:00,POINT (-85.24266 24.64134),-85.242658,24.641340,False,2024-07-06 03:55:19
8913,SENTINEL-2B,2024-07-06 03:55:34+00:00,POINT (-85.46745 25.52730),-85.467448,25.527303,False,2024-07-06 03:55:34
8914,SENTINEL-2B,2024-07-06 03:55:49+00:00,POINT (-85.69463 26.41298),-85.694633,26.412981,False,2024-07-06 03:55:49
8915,SENTINEL-2B,2024-07-06 03:56:04+00:00,POINT (-85.92435 27.29836),-85.924346,27.298362,False,2024-07-06 03:56:04
8916,SENTINEL-2B,2024-07-06 03:56:19+00:00,POINT (-86.15673 28.18343),-86.156726,28.183432,False,2024-07-06 03:56:19


Next, we can visualize these points on a map. They are color coded using a continuous color scale, with white being `now` and darker purple being further after `now`.

In [34]:
# This creates a range from 0-1 to define our colormap.
time_norm = (aoi.timestamp - t1) / (t2 - t1)

colors = apply_continuous_cmap(time_norm, Purples_9)

In [35]:
layer = ScatterplotLayer.from_geopandas(
    aoi,
    # extensions=[filter_extension],
    get_fill_color=colors,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m = Map(
    layer,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

We can clearly see the passes that Sentinel 2b takes in our area of interest and during our time frame.

---

## Calculating the position of a satellite constellation

Usually, humanitarians aren't concerned about which satellite captures an image, as long as they have a timely image with the appropriate resolution, (lack of) cloud cover, bands, etc.

The following example calculates the position of Planet satellites. With their portfolio's high spatial and temporal resolution, they are able to capture high-resolution images immediately before or after an event.

In [13]:
planet_url = "https://celestrak.org/NORAD/elements/gp.php?GROUP=planet&FORMAT=tle"

planet_swarm = load.tle_file(planet_url, reload=True, filename="planet_swarm")
planet_swarm[0]

[#################################] 100% planet_swarm


<EarthSatellite SKYSAT-A catalog #39418 epoch 2024-07-04 02:34:25 UTC>

We then calculate the locations for all Planet satellites during our time frame in set increments.

In [14]:
timer = t1

rows = []

while timer <= t2:
    for sat in planet_swarm:
        timer, longitude, latitude, coords, daytime = location_iteration(timer, sat)

        row = pd.DataFrame({'satellite': sat.name, 'timestamp': timer, 'coordinates': [coords], 'lng': longitude, 'lat': latitude, 'daytime': daytime}, index=[0])
        rows.append(row)    

    timer += timedelta(seconds=60) # ~ 10 min

planet_swarm_df = pd.concat(rows, ignore_index=True)
planet_swarm_df["time_string"] = planet_swarm_df["timestamp"].dt.strftime('%Y-%m-%d %X')


In [15]:
planet_swarm_gdf = gpd.GeoDataFrame(planet_swarm_df, geometry="coordinates")

Again, we want to look at the satellites in our AOI. Because Planet satellites capture optical images, the following shows the next passes that occur when the satellite is lit (as a proxy of when the ground below is lit).

In [16]:
mask_lon = (planet_swarm_gdf.lng >= min_lon) & (planet_swarm_gdf.lng <= max_lon)
mask_lat = (planet_swarm_gdf.lat >= min_lat) & (planet_swarm_gdf.lat <= max_lat)

aoi_planet = planet_swarm_gdf.where(mask_lon & mask_lat).dropna()
aoi_planet_day = aoi_planet[aoi_planet.daytime == True]
aoi_planet_day[aoi_planet_day["timestamp"] > now]

,satellite,timestamp,coordinates,lng,lat,daytime,time_string
1012,FLOCK 4Y-13,2024-07-04 14:53:19+00:00,POINT (-74.57982 23.13074),-74.579817,23.130736,True,2024-07-04 14:53:19
1749,FLOCK 4Y-4,2024-07-04 14:58:19+00:00,POINT (-74.81098 29.05651),-74.810978,29.056513,True,2024-07-04 14:58:19
1752,FLOCK 4Y-12,2024-07-04 14:58:19+00:00,POINT (-74.87942 28.02737),-74.879416,28.027366,True,2024-07-04 14:58:19
1899,FLOCK 4Y-4,2024-07-04 14:59:19+00:00,POINT (-75.68584 25.25064),-75.685845,25.250639,True,2024-07-04 14:59:19
1902,FLOCK 4Y-12,2024-07-04 14:59:19+00:00,POINT (-75.74340 24.22027),-75.743399,24.220275,True,2024-07-04 14:59:19
...,...,...,...,...,...,...,...
430064,SKYSAT-C13,2024-07-06 14:34:19+00:00,POINT (-78.12154 25.91856),-78.121541,25.918560,True,2024-07-06 14:34:19
430074,FLOCK 4S-11,2024-07-06 14:34:19+00:00,POINT (-79.60027 23.79144),-79.600273,23.791435,True,2024-07-06 14:34:19
430214,SKYSAT-C13,2024-07-06 14:35:19+00:00,POINT (-78.93753 22.08231),-78.937529,22.082311,True,2024-07-06 14:35:19
430655,SKYSAT-C2,2024-07-06 14:38:19+00:00,POINT (-86.97005 28.67295),-86.970051,28.672950,True,2024-07-06 14:38:19


In [36]:
# This creates a range from 0-1 to define our colormap.
time_norm_planet = (aoi_planet_day.timestamp - t1) / (t2 - t1)

colors_planet = apply_continuous_cmap(time_norm_planet, Purples_9)

Let's visualize this with the same color scheme we used before.

In [79]:
layer2 = ScatterplotLayer.from_geopandas(
    aoi_planet_day,
    # extensions=[filter_extension],
    get_fill_color=colors_planet,
    radius_min_pixels = 3
    # get_filter_value=filter_values,
    # filter_range=initial_filter_range,
)

m2 = Map(
    layer2,
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m2

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

As expected, there are many more passes than by a single satellite. Due to small area of interests for many events, cloud cover, and other variables, a larger constellation lets practitioners then narrow down their search.

In reality, satellites are taking more images than the amount of time steps that we arbitrarily set. A line would better represent the path these satellites take. The area of capture of satellites also varies, which a point doesn't represent.

Once a satellite is calculated to have passed over an AOI, the next step could be to analyze the imagery, ideally using the other STAC and COG focused notebooks in this repository. Combining a positioning calculation with automated image loading and processing could be the foundations for a powerful EO-based monitoring and alerting system.

---

In [45]:
paths = planet_swarm_gdf.groupby('satellite')['coordinates'].apply(lambda x: LineString(x.tolist()))
paths = gpd.GeoDataFrame(paths, geometry='coordinates')
paths = paths.reset_index()

In [48]:
viz([paths, aoi_planet_day])

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [77]:
layer3 = PathLayer.from_geopandas(
    paths,
    get_color=[255, 255, 255],
    width_scale=100,
    opacity=.5
)

/opt/homebrew/lib/python3.10/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [80]:
m3 = Map(
    [layer3, layer2],
    basemap_style = basemap.CartoBasemap.DarkMatter,
    )
m3

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…